<img src="https://imgur.com/K5IH3Ut.jpg" align="left" alt="Kitten" title="feup_logo" width="100" height="100" />




  #                                 FEUP - MIEIC - IART 2020/2021 





### Aplicação de "Reinforcement Learning" ao jogo "10Pair" 

Jogo original de onde foi adaptado: [10Pair](https://www.logicgamesonline.com/tenpair/)




### Definição do ambiente:

Mais simplificado do que o original.
Tamanho do tabuleiro 4 colunas por 2 linhas.
Sem opção de deal para aumentar mais o tabuleiro. Quando atinge um estado bloqueado o nivel dá reset.

Objetivo: Eliminar os números, esvaziando o tabuleiro
Os números são eliminados combinando-os se:
<ul>
<li>Forem números iguais</li>
<li>A soma dos números é 10</li>
<li>Adjacentes (vertical ou esquerda -> direita)</li>
<li>Espaços vazios (0) não contam </li>
</ul>

Representação dos estados é feita pelo número decimal coorrespondente ao número binário do atual tabuleiro em que se o número na posição for diferente de 0, coorresponde a um 1 e se a posição no tabuleiro for um 0, coorresponde a um 0 no número binário.

Exemplo:<br>
[2, 3, 3, 0]<br>
[8, 3, 7, 0]

numero binário: 11101110<br>
numero décimal: 238<br>

Por cada ação possivel executada que não resulte na resolução do nivel é dada uma reward de -1.
Se for feita uma ação que resulte na conclusão do nivel é dada uma reward de 1.
Se uma ação resultar num estado bloqueado em que não seja possível tomar nenhuma ação, é dada uma reward de -2

In [1]:
#create our custom environment for the game

import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np
import gym
import random
from copy import deepcopy

class game_10pair(gym.Env):
    
 board = np.zeros(shape=(2,4))

 #commented levels that gave us the most trouble 
 #board = np.array([[5, 5, 9, 4],[5, 4, 9, 5]])
 #board = np.array([[8, 1, 2, 1],[9, 8, 2, 1]])
 #board = np.array([[5, 9, 9, 9],[9, 1, 5, 9]])
    
 current_state = 0

 usable_board = np.array(board)
    
 def __init__(self):
        self.action_space = gym.spaces.Discrete(11)
        self.observation_space = gym.spaces.Discrete(256)
        
        i=0;
          
        #to use one of the hard coded levels coment the next line
        self.create_board()
        
        self.state = self.reset()
        
 def reverse_Action(self, quant):
    
        #0(numeros iguais) 1(soma=10)
        action = random.randint(0, 1)

        n1 = random.randint(1, 9)
        
        if action == 0:
            n2 = n1
        elif action ==1:
            n2 = 10-n1
            
        
        x1 = random.randint(0, 3)
        y1 = random.randint(0, 1)
        
        
        if quant == 3:
            f=1
            xi=0
            yi=0
            
            while 1:
                if self.board[yi][xi] == 0:
                    if f==1:
                        self.board[yi][xi]=n1                 
                        f=2
                    elif f==2:
                        self.board[yi][xi]=n2
                        return 1
                
                xi = xi+1
                
                if xi > 3:
                    xi = xi-4
                    yi = 1
                
        
        if self.board[y1][x1] == 0:
            
            #0(horizontal) 1(vertical)
            d = action = random.randint(0, 1)
           
            if d == 0:
                 
                x2 = random.randint(0,7)
                
                if x2 > 3:
                    x2=x2-4
                    y2=1
                else: 
                    y2=0
                if self.possible(x1, y1, x2, y2) == True and (x1 != x2 and y1 != y2):
                    
                    if self.board[y2][x2] == 0:

                        self.board[y1][x1] = n1
                        self.board[y2][x2] = n2
                        
                        return 1
                                    
            elif d==1:
                if y1 == 0 and self.board[y1+1][x1]==0:
                    y2 = 1
                    x2 = x1
                    self.board[y1][x1] = n1
                    self.board[y2][x2] = n2
                    
                    return 1
                elif y1 == 1 and self.board[y1-1][x1]==0:
                    y2 = 0
                    x2 = x1
                    self.board[y1][x1] = n1
                    self.board[y2][x2] = n2
                    
                    return 1
                
        return 0
    
 def create_board(self):
        count = 0

        while count < 4:
            count = count + self.reverse_Action(count)

 def possible(self, x1, y1, x2, y2):
            
        xi=0
        xo=0
        yi=0
        yo=0
        
        if y1<y2:
            xi = x1+1
            yi = y1
            xo = x2
            yo = y2
        elif y2<y1:
            xi = x2+1
            yi = y2
            xo = x1
            yo = y1
        elif y2==y1:
            if x1<x2:
                xi=x1+1
                yi = y1
                xo = x2
                yo = yi
            elif x2<x1:
                xi=x2+1
                yi = y2
                xo = x1
                yo = yi
        
        if xi > 3:
            xi = xi-4
            yi = 1
        
        while 1:
            
            if xi == xo and yi==yo:
                break
            
            if self.board[yi][xi] != 0:
                return False
        
            xi +=1
            
            if xi > 3 and yi == 0:
                xi=xi-4
                yi = 1
        
        return True
      
 def step(self, action):
        
        #if blocked
        if(action == 404):
            reward = -2
            state = self.state
            done = True
        else:
            first, second = self.action_to_move(action)

            self.usable_board[first[0]][first[1]]=0
            self.usable_board[second[0]][second[1]]=0

            done = self.game_ended()
            
            if done == True:
                reward = 1
            else:
                reward = -1

            board_aux = np.concatenate(self.usable_board)

            only1and0=[1 if i!=0 else 0 for i in board_aux]

            state = 0

            for i in range(len(only1and0)):
                state += only1and0[i]*2**(7-i)
            
            self.state = state
        return reward, done, state
    
 def get_possible_action(self):
    
    
    first = []
    second = []
    count = 0
    while count < 100:
        r = random.randint(0, 10)
        
        first, second = self.action_to_move(r)
        
        if(self.possible_move(first, second)):
            return r
        
        first.clear()
        second.clear()
        
        count = count + 1
    
    return 404
                       
 def possible_move(self, one, two):
               
    xi = one[1]
    yi = one[0]
    
    xo = two[1]
    yo = two[0]
    
    if self.usable_board[yi][xi] == 0 or self.usable_board[yo][xo] == 0:
        return False
    
    #same vertical
    if(xi == xo):
        if (self.usable_board[yi][xi] == self.usable_board[yo][xo]) or (self.usable_board[yi][xi]+self.usable_board[yo][xo] == 10):
            #self.usable_board[yi][xi] = 0
            #self.usable_board[yo][xo] = 0
            if(self.usable_board[yi][xi] != 0):
                return True
    else:#same horizontal
        board_aux = np.concatenate(self.usable_board)
        
        if yi == 1:
            xi = xi+4
        
        if yo == 1:
            xo = xo+4
            
        if(xi < xo):
            i=xi+1
            j=xo
        elif (xo<xi):
            i=xo+1
            j=xi
            
        sum = 0
        
        while i != j: 
            sum = sum + board_aux[i]
            i = i+1
            
        if ((board_aux[xi] == board_aux[xo]) or (board_aux[xi]+board_aux[xo] == 10)) and (sum == 0):
            #self.usable_board[init[0]][init[1]] = 0
            #self.usable_board[dest[0]][dest[1]] = 0
            return True
        
    return False
 def render(self):
    print(self.usable_board)

 def reset(self):
               
        self.usable_board = deepcopy(self.board)
        
        state = 255
        return state
 def game_ended(self):
    board_aux = np.concatenate(self.usable_board)
    
    if sum(board_aux) != 0:
        return False
    else:
        return True
     #functions that converts the action number to coordinates of numbers to remove
 def find_next(self, xi):
        self.board_aux = np.concatenate(self.usable_board)

        sol = []

        x = xi+1
        
        while x < len(self.board_aux):
            if self.board_aux[x] != 0:
                if x > 3:
                    sol.append(1)
                    sol.append(x-4)
                else: 
                    sol.append(0)
                    sol.append(x)
                    
                return sol
            elif x == 7:
                sol.append(1)
                sol.append(x-4)
                
                return sol
            
            x = x+1
        


 def action_to_move(self, action):
        #pair of numbers to remove
        num1 = [] 
        num2 = []

        if action == 0:
            num1.append(0) #y coord
            num1.append(0) #x coord

            aux = self.find_next(0)

            num2.append(aux[0])#y coord
            num2.append(aux[1])#x coord

            return num1, num2
        elif action == 1:
            num1.append(0) #y coord
            num1.append(1) #x coord

            aux = self.find_next(1)

            num2.append(aux[0])#y coord
            num2.append(aux[1])#x coord

            return num1, num2
        elif action == 2:
            num1.append(0) #y coord
            num1.append(2) #x coord

            aux = self.find_next(2)

            num2.append(aux[0])#y coord
            num2.append(aux[1])#x coord

            return num1, num2
        elif action == 3:
            num1.append(0) #y coord
            num1.append(3) #x coord

            aux = self.find_next(3)

            num2.append(aux[0])#y coord
            num2.append(aux[1])#x coord

            return num1, num2
        elif action == 4:
            num1.append(1) #y coord
            num1.append(0) #x coord

            aux = self.find_next(4)

            num2.append(aux[0])#y coord
            num2.append(aux[1])#x coord

            return num1, num2
        elif action == 5:
            num1.append(1) #y coord
            num1.append(1) #x coord

            aux = self.find_next(5)

            num2.append(aux[0])#y coord
            num2.append(aux[1])#x coord

            return num1, num2
        elif action == 6:
            num1.append(1) #y coord
            num1.append(2) #x coord

            num2.append(1) #y coord
            num2.append(3) #x coord

            return num1, num2
        elif action == 7:
            num1.append(0) #y coord
            num1.append(0) #x coord

            num2.append(1) #y coord
            num2.append(0) #x coord

            return num1, num2
        elif action == 8:
            num1.append(0) #y coord
            num1.append(1) #x coord

            num2.append(1) #y coord
            num2.append(1) #x coord

            return num1, num2
        elif action == 9:
            num1.append(0) #y coord
            num1.append(2) #x coord

            num2.append(1) #y coord
            num2.append(2) #x coord

            return num1, num2
        elif action == 10:
            num1.append(0) #y coord
            num1.append(3) #x coord

            num2.append(1) #y coord
            num2.append(3) #x coord

            return num1, num2



In [2]:
# Set hyperparameters for Q-learning

# @hyperparameters
total_episodes = 50        # Total episodes
learning_rate = 0.5          # Learning rate
max_steps = 99                # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1               # Exploration rate
max_epsilon = 1             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.001             # Exponential decay rate for exploration prob

In [3]:
#Criating the environment object
env = game_10pair()

print("Current board")
env.render()

# List of rewards
rewards = []

# Create Q-table
action_size = env.action_space.n
state_size = env.observation_space.n
print(f'action size: {action_size}, state size: {state_size}')

qtable = np.zeros((state_size, action_size))
print(qtable)

done = False

reward = 0

total_reward = 0

#env.render()

for episode in range(total_episodes):
    total_rewards = 0
    state = 255
    
    done = False
    
    env.reset()
    
    for steps in range(max_steps):
        
        exp_exp_tradeoff = random.uniform(0, 1)
        
        #print(f"exp_exp_tradeoff: {exp_exp_tradeoff}")
        
        ## If this number > greater than epsilon --> exploitation 
        #(taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            #print('exploit')

            mx_reward = -99
            mx_index = 0

            for i in range(11):
                if qtable[state, i] != 0 and qtable[state, i] > mx_reward and env.possible_move(*env.action_to_move(i)) == True:
                    mx_reward = qtable[state, i]
                    mx_index = i

            action = mx_index
                # Else doing a random choice --> exploration
        else:
            action = env.get_possible_action()
            
        #print('before step')
            
        reward, done, new_state = env.step(action)
        
        
        
        state_actions={}
        
    
        state_actions=qtable[new_state,:] 
            
        if sum(state_actions) == 0:
            new_state_mx_reward = 0
        else:
            #new_state_mx_reward = np.max(qtable[new_state, :])
            ma = state_actions[state_actions != 0]           
            new_state_mx_reward = ma.max()
          
        if action == 404:
                qtable[state, :] = qtable[state, :] + reward
        else:
            qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * new_state_mx_reward - qtable[state, action])
        
        total_reward = total_reward + reward
            
        state = new_state
        
        #game ended
        if done:
            
            
            break
   
         
        # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

#print(qtable)


Current board
[[4. 5. 6. 2.]
 [6. 5. 8. 4.]]
action size: 11, state size: 256
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
# Visualize learning outcome

env.reset()

state = []

board = np.concatenate(env.usable_board)

for i in range(len(qtable)):
    state = qtable[i,:] 
    
    if sum(state) != 0:
      
        bin_state = ([int(d) for d in str(bin(i))[2:].zfill(8)])
        
        state_board = []
        
        state_board=[board[j] if bin_state[j]!=0 else 0 for j in range(len(bin_state))]
        
        middle_index = len(state_board)//2            
        print(np.array([state_board[:middle_index], state_board[middle_index:]]))
        print(state) 
  
    state = []

# Print the action in every place
#Actions from 0 to 6 do horizontal combinations 
#pick the number on the action index and combine it the first one on its right diferent from 0
#Actions from 7 to 10 do vertical combinations
#where 7 in the action combining the first pair of vertical numbers and 10 refers to combining the last vertical pair

[[0. 0. 0. 2.]
 [6. 0. 8. 4.]]
[-24. -24. -24. -24. -24. -24. -24. -24. -24. -24. -24.]
[[0. 0. 6. 0.]
 [0. 0. 0. 4.]]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0. 0. 6. 2.]
 [0. 0. 8. 4.]]
[ 0.    0.    0.   -0.05  0.    0.    0.    0.    0.    0.    0.  ]
[[0. 5. 6. 2.]
 [0. 5. 8. 4.]]
[ 0.          0.          0.          0.          0.          0.
  0.          0.         -1.04750092  0.          0.        ]
[[4. 0. 6. 2.]
 [6. 0. 8. 4.]]
[-20.00068359   0.           0.           0.           0.
   0.           0.          -1.04750729   0.           0.
   0.        ]
[[4. 5. 6. 2.]
 [6. 5. 8. 4.]]
[ 0.          0.          0.          0.          0.          0.
  0.         -1.99513927 -1.99514108  0.          0.        ]


#### Na Qtable apresentada em cima, apenas estão representados os estados possiveis que possam existir no nível.

<ul>
<li>Uma reward=0 para uma ação representa um movimento impossível de se concretizar.
<li>Um estado em que todas as ações tenham rewards iguais e negativas corresponde a um estado em que o nivel está bloqueado e não é possível prosseguir com nenhum movimento, sendo necessário dar reset ao nível.
<li>Um estado tem uma ação com uma reward positiva quando esse movimento leva á conclusão de um nível.
</ul>

In [5]:
#Full Exploit testing!

#All the episodes are the same taking the maximum of Qtable value every time

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        env.render()
        
        #print(qtable[state, :])
        
        mx_reward = -99
        mx_index = 0
        
        for i in range(11):
            if qtable[state, i] != 0 and qtable[state, i] > mx_reward and env.possible_move(*env.action_to_move(i)) == True:
                mx_reward = qtable[state, i]
                mx_index = i

        action = mx_index
        
        print("action to perform: ", action)
        
        print("possible: ", env.possible_move(*env.action_to_move(action)))
        
        reward, done, new_state = env.step(action)
        
        
        
        if done:
            break
        state = new_state
    env.render()
env.close()

****************************************************
EPISODE  0
[[4. 5. 6. 2.]
 [6. 5. 8. 4.]]
action to perform:  7
possible:  True
[[0. 5. 6. 2.]
 [0. 5. 8. 4.]]
action to perform:  8
possible:  True
[[0. 0. 6. 2.]
 [0. 0. 8. 4.]]
action to perform:  3
possible:  True
[[0. 0. 6. 0.]
 [0. 0. 0. 4.]]
action to perform:  2
possible:  True
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
****************************************************
EPISODE  1
[[4. 5. 6. 2.]
 [6. 5. 8. 4.]]
action to perform:  7
possible:  True
[[0. 5. 6. 2.]
 [0. 5. 8. 4.]]
action to perform:  8
possible:  True
[[0. 0. 6. 2.]
 [0. 0. 8. 4.]]
action to perform:  3
possible:  True
[[0. 0. 6. 0.]
 [0. 0. 0. 4.]]
action to perform:  2
possible:  True
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
****************************************************
EPISODE  2
[[4. 5. 6. 2.]
 [6. 5. 8. 4.]]
action to perform:  7
possible:  True
[[0. 5. 6. 2.]
 [0. 5. 8. 4.]]
action to perform:  8
possible:  True
[[0. 0. 6. 2.]
 [0. 0. 8. 4.]]
action to perform:  3
pos

## Lista de ações

<ul>
<li>0: movimento para a direita do primeiro número da primeira linha</li>
<li>1: movimento para a direita do segundo número da primeira linha</li>
<li>2: movimento para a direita do terceiro número da primeira linha</li>
<li>3: movimento para a direita do quarto número da primeira linha</li>
<li>4: movimento para a direita do primeiro número da segunda linha</li>
<li>5: movimento para a direita do segundo número da segunda linha</li>
<li>6: movimento para a direita do terceiro número da segunda linha</li>
<li>7: movimento vertical na primeira coluna</li>
<li>8: movimento vertical na segunda coluna</li>
<li>9: movimento vertical na terceira coluna</li>
<li>10: movimento vertical na quarta coluna</li>
</ul>